#  LangChain의 개념과 주요 컴포넌트 이해

---

## 1. LangChain 소개

- **LangChain**은 대화형 AI 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임워크입니다.

- **핵심 가치**
    - **모듈화**: 독립적인 컴포넌트를 조합하여 복잡한 AI 시스템 구축
    - **상호운용성**: 다양한 AI 모델과 데이터 소스를 하나의 인터페이스로 통합
    - **확장성**: 간단한 챗봇부터 복잡한 AI 에이전트까지 확장 가능
    - **관찰성**: LangSmith를 통한 실시간 모니터링 및 디버깅

- **핵심 아키텍처**
    ```markdown
    ├── langchain-core     # 기본 추상화 및 인터페이스
    ├── langchain          # 체인, 에이전트, 검색 전략
    ├── langchain-openai   # OpenAI 통합
    ├── LangGraph          # 복잡한 에이전트 워크플로우
    └── LangSmith          # 모니터링 및 디버깅
    ```

<div style="text-align: center;">
    <img src="https://python.langchain.com/svg/langchain_stack_112024_dark.svg" 
        alt="langchain_stack" 
        width="600" 
        style="border: 0;">
</div>

- **LangSmith 모니터링**

    - **LangSmith**는 LLM 애플리케이션의 관찰성(Observability)을 제공하는 도구입니다.

    - **주요 기능**
        - **체인 실행 로깅 및 추적**
        - **프롬프트 디버깅**
        - **성능 측정 및 분석**
        - **실시간 모니터링**

    - **계정 가입 및 설정**
        ```python
        # 1. LangSmith 계정 가입: https://www.langchain.com/langsmith
        # 2. .env 파일 설정
        LANGSMITH_API_KEY=your_langsmith_api_key
        LANGSMITH_TRACING=true
        LANGSMITH_PROJECT=your_project_name

        # 3. 환경 확인
        from dotenv import load_dotenv
        import os

        load_dotenv()
        print(f"LangSmith 추적 상태: {os.getenv('LANGSMITH_TRACING')}")
        print(f"프로젝트명: {os.getenv('LANGSMITH_PROJECT')}")
        ```

---

## 2. 환경 설정

- **설치**

    ```bash
    # pip 설치
    pip install langchain langchain-openai langchain-google-genai

    # uv 설치 
    uv add langchain langchain-openai langchain-google-genai

    # 추가 도구 pip 설치 (선택사항)
    pip install langchain-ollama langsmith

    # 추가 도구 uv 설치 (선택사항)
    uv add langchain-ollama langsmith
    ``` 

- **API 키 설정**

    ```python
    # .env 파일 생성
    OPENAI_API_KEY=your_openai_api_key_here
    GOOGLE_API_KEY=your_google_api_key_here

    # LangSmith 설정 (선택사항)
    LANGSMITH_API_KEY=your_langsmith_api_key
    LANGSMITH_TRACING=true
    LANGSMITH_PROJECT=your_project_name
    ```

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# LangSmith 추적 확인
import os
print(f"LangSmith 추적: {os.getenv('LANGSMITH_TRACING')}")

---

## 3. 핵심 컴포넌트

### 3.1 Chat Models (채팅 모델)

- OpenAI, Anthropic, Google 등 다양한 모델을 지원
- 텍스트 생성, 대화, 요약 등의 작업을 수행

In [ ]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3,
    top_p=0.95
)

# 간단한 대화
response = model.invoke("탄소의 원자 번호는 무엇인가요?")

In [ ]:
print(f"답변: {response.content}")

In [ ]:
print(f"메타데이터: {response.response_metadata}")

### 3.2 Messages (메시지)

- 메시지는 AI와의 대화에서 역할을 구분하는 기본 단위입니다.
- 메시지는 사용자, AI, 시스템 등 다양한 역할을 가질 수 있습니다.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 시스템 메시지: AI의 역할 정의
system_msg = SystemMessage(content="당신은 친절한 화학 선생님입니다.")

# 사용자 메시지
human_msg = HumanMessage(content="탄소의 원자 번호는 몇 번인가요?")

# 대화 실행
messages = [system_msg, human_msg]
response = model.invoke(messages)

In [ ]:
print(response.content)

In [ ]:
from pprint import pprint
pprint(response.response_metadata)

### 3.3 Prompt Templates (프롬프트 템플릿)

- 템플릿을 사용하여 일관된 프롬프트를 생성할 수 있습니다.
- 변수 치환을 통해 동적인 프롬프트를 적용하는 데 유용합니다.


`(1) 기본 템플릿`

In [ ]:
from langchain_core.prompts import PromptTemplate

# 전문가 템플릿
template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.
질문: {question}
"""

prompt = PromptTemplate.from_template(template)

# 템플릿 입력 변수 확인
print(f"필수 변수: {prompt.input_variables}")

In [ ]:
# 템플릿 확인
print(f"템플릿: {prompt.template}")

In [ ]:
# 템플릿 사용
formatted_prompt = prompt.format(
    topic="화학",
    question="탄소의 원자 번호는 무엇인가요?"
)
print(formatted_prompt)

In [ ]:
# LLM 모델에 전달하여 답변 생성
response = model.invoke(formatted_prompt)

In [ ]:
print(response.content)

`(2) 채팅 템플릿`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅용 템플릿
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 {subject} 상담사입니다."),
    ("human", "{question}")
])

# 템플릿 사용
prompt = chat_template.format_messages(
    subject="진로",
    question="데이터 사이언티스트가 되려면 어떤 공부를 해야 하나요?"
)

In [ ]:
# 템플릿 확인
pprint(prompt)

In [ ]:
# LLM 모델에 전달하여 답변 생성
response = model.invoke(prompt)

In [ ]:
print(response.content)

---

## 4. LCEL (LangChain Expression Language)

### 4.1 LCEL이란?

- **LCEL**은 `|` 연산자를 사용하여 컴포넌트들을 순차적으로 연결하는 선언적 체이닝을 지원

- **핵심 특징**
    - **재사용성**: 정의된 체인을 다른 체인의 컴포넌트로 활용
    - **다양한 실행 방식**: `.invoke()`, `.batch()`, `.stream()`, `.astream()`
    - **자동 최적화**: 배치 처리 시 효율적인 작업 수행
    - **스키마 지원**: 입력/출력 스키마 자동 생성

### 4.2 기본 체인 구성

`(1) Prompt + LLM`

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 컴포넌트 정의
prompt = PromptTemplate.from_template(
    "당신은 {topic} 분야의 전문가입니다. {topic}에 관한 다음 질문에 답변해주세요.\n"
    "질문: {question}"
)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 체인 구성
chain = prompt | llm

# 체인 실행
response = chain.invoke({
    "topic": "화학",
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print(f"답변: {response.content}")  # AIMessage 객체로 반환

`(2) Prompt + LLM + Output Parser`

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 추가
output_parser = StrOutputParser()

# 완전한 체인 구성
chain = prompt | llm | output_parser

# 체인 실행 (문자열 반환)
response = chain.invoke({
    "topic": "화학",
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print(f"답변: {response}")  # 문자열로 반환

---

## 5. Runnable

* 실행 인터페이스: 모든 LangChain 컴포넌트는 Runnable 인터페이스를 구현하여 일관된 방식으로 실행

* 실행 메서드: `.invoke()`, `.batch()`, `.stream()`, `.astream()` 등 다양한 실행 방식을 제공

* 호환성: 모든 Runnable 컴포넌트는 파이프(|) 연산자를 통해 연결 가능하며, 재사용이 용이

* Runnable의 주요 유형:

    * `RunnableSequence`: 여러 Runnable을 순차적으로 실행
    * `RunnablePassthrough`: 입력을 그대로 다음 단계로 전달    
    * `RunnableParallel`: 여러 Runnable을 병렬로 실행
    * `RunnableLambda`: 파이썬 함수를 Runnable로 래핑하여 체인에서 사용

### 5.1 RunnableSequence (순차 실행)

- **RunnableSequence**는 컴포넌트들을 연결하여 순차적으로 데이터를 처리하는 체인

- `|` 연산자로 연결된 각 단계의 **출력이 다음 단계의 입력**으로 전달

- **다양한 실행 방식**(동기/비동기, 배치/스트리밍)을 지원

- LLM 체인, 데이터 파이프라인, 자동화된 작업 등 **다단계 처리**에 활용

In [ ]:
# 번역 전용 체인 (파이프 연산자 사용)
translation_prompt = PromptTemplate.from_template(
    "'{text}'를 영어로 번역해주세요. 번역된 문장만을 출력해주세요."
)

translation_chain = translation_prompt | llm | StrOutputParser()

# 번역 실행
result = translation_chain.invoke({"text": "좋은 하루 되세요!"})
print(result)

In [ ]:
from langchain_core.runnables import RunnableSequence

# 명시적 RunnableSequence 생성
translation_chain = RunnableSequence(
    first=translation_prompt, 
    middle=[llm],
    last=StrOutputParser()
)

# 파이프 연산자와 동일한 결과
# translation_chain = translation_prompt | llm | StrOutputParser()

result = translation_chain.invoke({"text": "좋은 하루 되세요!"})
print(result)

### 5.2 RunnableLambda (함수 래핑)

- **RunnableLambda**는 일반 함수를 Runnable 객체로 변환하는 래퍼 컴포넌트

- 체인에 **커스텀 로직**을 쉽게 통합할 수 있어 데이터 전처리, 후처리에 유용

- `|` 연산자로 다른 컴포넌트들과 연결해 **복잡한 처리 흐름**을 구성 가능

In [ ]:
import re
from langchain_core.runnables import RunnableLambda

# 텍스트에서 숫자를 추출하는 함수
def extract_number(query):
    return int(re.search(r'\d+', query).group())

# RunnablePassthrough로 입력을 그대로 전달하고, RunnableLambda로 숫자 추출 함수 실행
runnable = RunnableLambda(extract_number)

# 입력 텍스트에서 6을 추출
result = runnable.invoke('탄소의 원자 번호는 6입니다.')
print(result)

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage

# 데이터 전처리 함수 정의
def preprocess_text(text: str) -> str:
    """ 입력 텍스트를 소문자로 변환하고 양쪽 공백을 제거합니다. """
    return text.strip().lower()

# 후처리 함수 정의
def postprocess_response(response: AIMessage) -> dict:
    """ 응답 텍스트를 대문자로 변환하고 길이를 계산합니다. """
    response_text = response.content
    return {
        "processed_response": response_text.upper(),
        "length": len(response_text)
    }

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("다음 주제에 대해 영어 한 문장으로 설명해주세요: {topic}")

# 처리 파이프라인 구성
chain = (
    RunnableLambda(preprocess_text) |  # 입력 전처리
    prompt |                           # 프롬프트 포맷팅
    llm |                              # LLM 추론
    RunnableLambda(postprocess_response)  # 출력 후처리
)

# 체인 실행
result = chain.invoke("  Artificial Intelligence  ")

In [ ]:
print(f"응답 길이: {result['length']}")

In [ ]:
print(f"처리된 응답: {result['processed_response']}")

### 5.3 RunnableParallel (병렬 실행)

- **RunnableParallel**은 여러 컴포넌트를 딕셔너리 형태로 구성해 **동시 실행**

- 동일한 입력이 모든 병렬 컴포넌트에 전달되며, 결과는 **키-값 쌍**으로 반환

- **데이터 변환**과 **파이프라인 구성**에 특화되어 있으며, 출력 형식을 다음 단계에 맞게 조정 가능

`(1) 질문 분석 체인`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

# Pydantic 모델로 구조화된 출력 정의
class SubjectClassification(BaseModel):
    """질문의 주제 분류 결과"""
    category: Literal["화학(Chemistry)", "물리(Physics)", "생물(Biology)"] = Field(
        description="질문이 속하는 과학 분야 카테고리"
    )
    reasoning: str = Field(
        description="분류 이유에 대한 짧은 설명"
    )

# 프롬프트 템플릿 정의
classification_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 질문을 과학 분야로 분류하는 전문가다.
        질문을 분석하고 해당하는 카테고리로 정확하게 분류해야 한다.
        
        분류 기준:
        - 화학(Chemistry): 원소, 화합물, 반응, 분자 구조 등
        - 물리(Physics): 힘, 운동, 에너지, 파동, 전기 등  
        - 생물(Biology): 생명체, 세포, 유전, 생태계 등"""
    ),
    ("human", "{question}")
])

# 구조화된 출력을 사용한 체인 구성
structured_llm = llm.with_structured_output(SubjectClassification)
classification_chain = classification_prompt | structured_llm

# 사용 예시
result = classification_chain.invoke({"question": "탄소의 원자 번호는 무엇인가요?"})

# 결과는 Pydantic 객체로 반환됨
print(f"분류: {result.category}")
print(f"이유: {result.reasoning}")

`(2) 언어 감지 체인`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

# Pydantic 모델로 구조화된 출력 정의
class LanguageDetection(BaseModel):
    """텍스트의 언어 감지 결과"""

    detected_language: Literal["영어(English)", "한국어(Korean)", "스페인어(Spanish)", "중국어(Chinese)", "일본어(Japanese)", "기타(Others)"] = Field(
         description="질문에 사용된 메인 언어"
    )    
    explanation: str = Field(
        description="언어 감지 근거에 대한 짧은 설명"
    )

# 프롬프트 템플릿 정의
language_detection_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 언어 감지 전문가다. 
        주어진 텍스트의 언어를 정확하게 식별하고 분석해야 한다.
        
        다음 언어들을 주로 구분한다:
        - English: 영어
        - Korean: 한국어
        - Spanish: 스페인어
        - Chinese: 중국어
        - Japanese: 일본어
        - Others: 기타 언어
        
        텍스트의 문자 체계, 단어 패턴, 문법 구조를 분석하여 언어를 판별한다."""
    ),
    ("human", "다음 텍스트의 언어를 감지하세요: {question}")
])

# 구조화된 출력을 사용한 체인 구성
structured_llm = llm.with_structured_output(LanguageDetection)
language_chain = language_detection_prompt | structured_llm

In [ ]:
# 사용 예시
examples = [
    "What is the atomic number of carbon?",
    "탄소의 원자 번호는 무엇인가요?",
    "碳的原子序数是多少？",
    "炭素の原子番号は何ですか？",
]

# 각 예시 처리
for example in examples:
    result = language_chain.invoke({"question": example})
    
    print(f"\n입력: {example}")
    print(f"언어: {result.detected_language}")
    print(f"설명: {result.explanation}")
    print(f"--------------------------------")

`(3) RunnableParallel을 사용한 병렬 실행 체인`

In [ ]:
# 질문과 관련된 분야를 찾아서 질문에 대한 답변을 생성하는 프롬프트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# 답변 템플릿 정의
answer_template = """
당신은 {topic} 분야의 전문가입니다. {topic}에 관한 질문에 {language}로 답변해주세요.
질문: {question}
"""

# 프롬프트 및 체인 구성
answer_prompt = PromptTemplate.from_template(answer_template)
output_parser = StrOutputParser()

# 병렬 처리 체인 구성
answer_chain = RunnableParallel({
    "topic": classification_chain | RunnableLambda(lambda x: x.category),            # 주제 분류 체인
    "language": language_chain | RunnableLambda(lambda x: x.detected_language),         # 언어 감지 체인
    "question": itemgetter("question")  # 원본 질문 추출
}) | answer_prompt | llm | output_parser

# 체인 실행 예시
result = answer_chain.invoke({
    "question": "탄소의 원자 번호는 무엇인가요?"
})

print("처리 결과:")
print(f"답변: {result}")

### 5.4 RunnablePassthrough (입력을 출력으로 그대로 전달)

- **RunnablePassthrough**는 입력값을 그대로 전달하여 원본 데이터를 보존

- **RunnableParallel**과 함께 사용되어 입력 데이터를 새로운 키로 매핑 가능

- **투명한 데이터 흐름**으로 파이프라인 디버깅과 구성이 용이

In [ ]:
from langchain_core.runnables import RunnablePassthrough
import re

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x["query"]).group()),
)

runnable.invoke({"query": '탄소의 원자 번호는 6입니다.'})

In [ ]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: int(re.search(r'\d+', x).group()),
)

runnable.invoke('탄소의 원자 번호는 6입니다.')

---

## 6. 실습 문제

#### Gradio ChatInterface  
- 설치: uv add gradio

`(1) 기본 구조`

In [ ]:
import gradio as gr

# 챗봇 함수 정의
def chat_function(message, history):
    return message

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=chat_function,  # 실행할 함수
    analytics_enabled=False,  # 사용 정보 제공 여부
    type="messages",
)

# 챗봇 인터페이스 실행
demo.launch()

In [ ]:
# 인터페이스 종료
demo.close()

`(2) 문제: RunnableParallel 활용`
- 하나의 입력에 대해 번역과 요약을 동시에 수행하는 체인을 만드세요.
- gradio 인터페이스를 활용하여 챗봇 형태로 구현하세요.


In [ ]:
# 여기에 코드를 작성하세요
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from gradio import ChatInterface


<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from gradio import ChatInterface

# 모델 정의
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)

# 번역 체인
translate_prompt = PromptTemplate.from_template(
    "다음 텍스트를 영어로 번역하세요: {text}"
)
translate_chain = translate_prompt | model | StrOutputParser()

# 요약 체인  
summarize_prompt = PromptTemplate.from_template(
    "다음 텍스트를 한 문장으로 요약하세요: {text}"
)
summarize_chain = summarize_prompt | model | StrOutputParser()

# 병렬 처리 체인
parallel_chain = RunnableParallel({
    "translation": translate_chain,
    "summary": summarize_chain
})

# 테스트
result = parallel_chain.invoke({
    "text": "안녕하세요. 오늘은 정말 좋은 날씨입니다. 산책하기 딱 좋네요."
})
print(f"번역: {result['translation']}")
print(f"요약: {result['summary']}")




def chat_function(message, history):

    result = parallel_chain.invoke({"text": message})
    translation = result['translation']
    summary = result['summary']

    return f"*번역*\n{translation}\n\n*요약*\n{summary}"

    
demo = ChatInterface(
    fn=chat_function,
    title="번역 및 요약 챗봇",
    description="원하는 텍스트를 입력하면 번역과 요약을 동시에 제공합니다.",
    analytics_enabled=False,
    type="messages",
    examples=[
        "안녕하세요. 오늘은 정말 좋은 날씨입니다. 산책하기 딱 좋네요.",
    ]
)
```
</details>
